In [ ]:
import numpy as np
import scipy
import datetime as dt
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pickle
from multiprocessing import Pool
import os
import Rheology_fitting_toolkit as rft


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)



dictionary_all_events = {}
CREEPMETER = ['XHR']

for q in range(len(CREEPMETER)):
    print(CREEPMETER[q])
    tm, min10_creep, tm2, min10_creep2 = rft.import_text(CREEPMETER[q])

    if CREEPMETER[q] == 'XSJ' or CREEPMETER[q] == 'XHR' or CREEPMETER[q] == 'XPK':
        tm_int, min10_creep_int = rft.interpolate(tm,min10_creep,CREEPMETER)
        tm_int2, min10_creep_int2 = rft.interpolate(tm2,min10_creep2,CREEPMETER)
    elif CREEPMETER[q] == 'XMR':
        tm_int, min10_creep_int = rft.interpolate(tm,min10_creep,CREEPMETER)
        tm_int2, min10_creep_int2 = rft.interpolate_1min(tm2,min10_creep2,CREEPMETER)
    else:
        tm_int, min10_creep_int = rft.interpolate(tm,min10_creep,CREEPMETER)


    df_PICKS, duration, START = rft.creepmeter_events(CREEPMETER[q])

    if CREEPMETER[q] == 'XSJ' or CREEPMETER[q] == 'XHR' or CREEPMETER[q] == 'XPK':
        data1  = rft.vel_acc(tm_int,min10_creep_int,10/60)
        data2 = rft.vel_acc(tm_int2,min10_creep_int2,10/60)
        data = data1.append(data2,ignore_index=True)
    elif CREEPMETER[q] == 'XMR':
        data1  = rft.vel_acc(tm_int,min10_creep_int,10/60)
        data2 = rft.vel_acc_1min(tm_int2,min10_creep_int2,1/60)
        data = data1.append(data2,ignore_index=True)
    else:
        data = rft.vel_acc(tm_int,min10_creep_int,10/60)


    df_auto = rft.parkfield_remover(df_PICKS,CREEPMETER[q])


    df_rain_day_total = rft.rain_timeseries(CREEPMETER[q])

    df_auto = rft.when_does_it_rain(df_auto,CREEPMETER[q])
    
    if CREEPMETER[q] == 'CWN':
        C_matrix = np.load('../../Rheology/CWN/CWN_covariance_matrix_12days_18_APR_23.npy')
        C_matrix_inv_CWN = np.linalg.inv(C_matrix)
    
    if CREEPMETER[q] == 'XHR':
        C_matrix_2 = np.load('../../Rheology/XHR/XHR_2_covariance_matrix_4days_27_APR_23.npy')
        C_matrix_3 = np.load('../../Rheology/XHR/XHR_3_covariance_matrix_4days_27_APR_23.npy')
        C_matrix_inv_2 = np.linalg.inv(C_matrix_2)
        C_matrix_inv_3 = np.linalg.inv(C_matrix_3)
    
    dataframes_long, creep_index_long = rft.creep_event_dataframe(df_auto,duration, START, data,CREEPMETER[q])
    dataframes, creep_index = rft.creep_event_dataframe_short(dataframes_long,df_auto)

    Creep_phases = pd.read_csv("../../Rheology/{k}/Creep_phases_{k}.csv".format(k=CREEPMETER[q]),index_col=0)
    



In [ ]:
def VSF_aSS_fitter(time,slip,cov_matrix_inv,no_phases,columns_VSF_aSS,j,CREEPMETER,VSF_aSS_DF_params,atest,file_misfit):
    """
    Fit the VSF_aSS rheological model to slip data using basin-hopping optimization.

    This function attempts to fit the Velocity Strengthening Friction with aSS (VSF_aSS) 
    model to the provided slip data for a given event index. It checks if a fit dictionary 
    file exists for the event; if not, it performs optimization with updated initial 
    parameters and bounds for the 't1' parameter, repeatedly calling basin-hopping 
    until a successful fit is found. The fit results and parameters are saved to disk. 
    If the fit dictionary exists, it loads the stored results.

    Args:
        time (array-like): Time data points for the event.
        slip (array-like): Slip data corresponding to the time points.
        cov_matrix_inv (array-like): Inverse covariance matrix for weighting data.
        no_phases (int): Number of creep phases in the event.
        columns_VSF_aSS (list of str): Column names corresponding to VSF_aSS parameters.
        j (int): Event index.
        CREEPMETER (str): Identifier for the creepmeter station.
        VSF_aSS_DF_params (pandas.DataFrame): DataFrame containing initial guesses and bounds for parameters.
        atest (callable): Acceptance test function for basin-hopping.
        file_misfit (file-like object): File handle for logging misfit during fitting.

    Returns:
        pandas.DataFrame: Updated DataFrame with fitted VSF_aSS parameters.
    """
    print('VSF_aSS: {k}'.format(k=j))
    print ("Current date and time : ")
    print (dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    isExistVSF_aSS = os.path.exists('../../Rheology/Single_rheology_28_APR_23/{k}/fits/SLSQP/{y}/VSF_aSS/{k}_{y}_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt'.format(k=CREEPMETER,y=j))
    if not isExistVSF_aSS:
        success = False
        n_iter = 5000
        VSF_aSS_DF_params['t1'].loc['initial'] = 2
        VSF_aSS_DF_params['t1'].loc['bounds'] = (-10,10)
        VSF_aSS_bounds = VSF_aSS_DF_params.loc['bounds'].to_list()
        VSF_aSS_initial_guess = VSF_aSS_DF_params.loc['initial'].to_list()
        
        while success == False:
            if success == False:
                res_VSF_aSS = scipy.optimize.basinhopping(rft.VSF_aSS_dromedary, VSF_aSS_initial_guess,\
                accept_test = atest, minimizer_kwargs = ({'args':(time,slip,cov_matrix_inv,no_phases,\
                columns_VSF_aSS,j,CREEPMETER,file_misfit),'method':'SLSQP','bounds':VSF_aSS_bounds}))#,'options':{'maxiter':n_iter}}),niter=1000)
                n_iter = n_iter+2000
                success = res_VSF_aSS.success
                VSF_aSS_initial_guess = res_VSF_aSS.x
        dictionary_VSF_aSS = {}
        dictionary_VSF_aSS['fit'] = res_VSF_aSS
        VSF_aSS_fitting_params = pd.DataFrame([res_VSF_aSS.x],columns = ('Ts','Vs','K','T01','S1','Ta1','V01','t1','T02','S2'), index = ['fitted'])
        VSF_aSS_DF_params = pd.concat([VSF_aSS_DF_params,VSF_aSS_fitting_params])
        dictionary_VSF_aSS['fitting params'] = VSF_aSS_DF_params
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/SLSQP/{u}/VSF_aSS/{k}_{u}_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt".format(k=CREEPMETER,u=j), "wb") as tf:
            pickle.dump(dictionary_VSF_aSS,tf)

    else:
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/SLSQP/{u}/VSF_aSS/{k}_{u}_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt".format(k=CREEPMETER,u=j), "rb") as tf:
            dictionary_VSF_aSS = pickle.load(tf)
            res_VSF_aSS = dictionary_VSF_aSS['fit']
            VSF_aSS_DF_params = dictionary_VSF_aSS['fitting params']
    return VSF_aSS_DF_params

In [ ]:
def test_multi(j):
    """
    Run rheological model fitting and generate fit files and plots for a specific event index.

    This function processes slip-time data for the event at index `j` from a collection of
    long-duration dataframes. It checks if the processed results and plots already exist
    on disk to avoid redundant computations. If not, it performs phase splitting and
    initializes parameter bounds for fitting the VSF_aSS rheological model. It then runs
    the fitting procedure, saves the results, and prepares directories as needed.

    The function focuses on events with up to 577 time points and processes only if the
    number of creep phases is 1 or 2. It adjusts parameters and inverse covariance matrix
    based on creepmeter and event index.

    Args:
        j (int): Index of the event in the dataset to process.

    Returns:
        None: This function operates through side effects (file I/O, printing) and does not return a value.
    """
    print(j)
    if len(dataframes_long[j].Time) <= 577:
        isExist = os.path.exists('/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/figures/SLSQP/{y}/{k}_{y}_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.pdf'.format(k=CREEPMETER[q],y=j))
        if not isExist:
            isExist2 = os.path.exists('/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/SLSQP/{y}/{k}_{y}_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt'.format(k=CREEPMETER[q],y=j))
            if not isExist2:
                print('/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/SLSQP/{y}/{k}_{y}_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt'.format(k=CREEPMETER[q],y=j))
                Creep_Phase_no = Creep_phases.iloc[j].dropna()
                number_of_phases = (len(Creep_Phase_no)/2)-1
                if 1<= number_of_phases <=2:
                    data_P0, data_P1, data_P2, data_P3, data_P4, creep_phase_new = rft.phase_splitter(Creep_Phase_no,dataframes_long[j])

                    VSF_aSS_DF_params = rft.initial_and_bounds(creep_phase_new,data_P0,data_P1,data_P2,data_P3,data_P4,'VSF_aSS')
                    if CREEPMETER[q] == 'XHR':
                        if j < 70:
                            C_matrix_inv = C_matrix_inv_2
                        else:
                            C_matrix_inv = C_matrix_inv_3
                    else:
                        C_matrix_inv = C_matrix_inv_CWN

                    rft.check_dir("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}".format(k=CREEPMETER[q]))
                    Rheologies_to_test = ['LNV','PLV','VSF_SS','VSF_bSS','VSF_aSS','RDF']#,'cb77']
                    if number_of_phases==1:
                        VSF_aSS_DF_params.drop(['Ta2','V02','t2','T03','S3','Ta3','V03','t3','T04','S4','Ta4','V04','t4'], axis=1, inplace=True)

                        for z in range(len(Rheologies_to_test)):
                            rft.check_dir("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/SLSQP/{u}/{x}".format(k=CREEPMETER[q],u=j,x=Rheologies_to_test[z]))

                        VSF_aSS_params_tried = np.zeros(len(VSF_aSS_DF_params.loc['initial'].index.tolist()))
                        
                        columns_VSF_aSS = VSF_aSS_DF_params.loc['initial'].index.tolist()


                        rft.check_dir("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/SLSQP/{u}".format(k=CREEPMETER[q],u=j))

                        def atest(f_new, x_new, f_old, x_old):
                            if f_old < f_new:
                                return False
                            else:
                                return True

                        
                        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/SLSQP/{u}/VSF_aSS/{k}_{u}_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt".format(k=CREEPMETER[q],u=j),'a') as file_VSF_aSS:
                            VSF_aSS_DF_params = VSF_aSS_fitter(np.array(dataframes_long[j].Time),np.array(dataframes_long[j].Slip), C_matrix_inv,number_of_phases,columns_VSF_aSS,j,CREEPMETER[q],\
                                                               VSF_aSS_DF_params,atest,file_VSF_aSS)

    return

In [ ]:
dictionary_all_events = {}
if __name__ == '__main__':
    """
    Use multiprocessing to run `test_multi` on each creep event in `dataframes`.
    Currently uses a single worker process (Pool(1)), but can be adjusted for parallelism.
    """
    with Pool(1) as pool:                         # Create a multiprocessing Pool           
        pool.map(test_multi, dataframes)

In [ ]:
def VSF_aSS_fitter(time,slip,cov_matrix_inv,no_phases,columns_VSF_aSS,j,CREEPMETER,VSF_aSS_DF_params,atest,file_misfit):
    """
    Fit the VSF_aSS rheological model to slip-time data using basin-hopping optimization.

    This function attempts to load a previously saved Nelder-Mead fit result from disk.
    If not found, it initializes fitting parameters from a prior SLSQP fit, then performs
    a global optimization with the basin-hopping algorithm using Nelder-Mead as the local
    minimizer. Results are saved to disk as a pickle file.

    Args:
        time (np.array): Time data array.
        slip (np.array): Slip data array.
        cov_matrix_inv (np.array): Inverse covariance matrix used in misfit calculation.
        no_phases (int): Number of creep phases to fit.
        columns_VSF_aSS (list): List of parameter names for the VSF_aSS model.
        j (int): Index of the current event/dataframe being processed.
        CREEPMETER (str): Identifier for the creepmeter dataset.
        VSF_aSS_DF_params (pd.DataFrame): DataFrame with initial parameters and bounds.
        atest (callable): Acceptance test function for basin-hopping.
        file_misfit (file-like): File handle for writing misfit information (used in fitting).

    Returns:
        pd.DataFrame: Updated DataFrame of fitted VSF_aSS parameters.
    """
    print('VSF_aSS: {k}'.format(k=j))
    print ("Current date and time : ")
    print (dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    isExistVSF_aSS = os.path.exists('../../Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{y}/VSF_aSS/{k}_{y}_VSF_aSS_t1_neg_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt'.format(k=CREEPMETER,y=j))
    if not isExistVSF_aSS:
        success = False
        n_iter = 5000
        VSF_aSS_DF_params['t1'].loc['bounds'] = (-10,10)
        VSF_aSS_bounds = VSF_aSS_DF_params.loc['bounds'].to_list()
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/SLSQP/{u}/VSF_aSS/{k}_{u}_VSF_aSS_fit_dictionary_multi_phase_SLSQP_t1_neg.txt".format(k=CREEPMETER,u=j),'rb') as fname:
            VSF_aSS_SLSQP = pickle.load(fname)        
        VSF_aSS_initial_guess = VSF_aSS_SLSQP['fitting params'].loc['fitted'].to_list() 
        while success == False:
            if success == False:
                res_VSF_aSS = scipy.optimize.basinhopping(rft.VSF_aSS_dromedary, VSF_aSS_initial_guess,\
                accept_test = atest, minimizer_kwargs = ({'args':(time,slip,cov_matrix_inv,no_phases,\
                columns_VSF_aSS,j,CREEPMETER,file_misfit),'method':'Nelder-Mead','bounds':VSF_aSS_bounds}))#,'options':{'maxiter':n_iter}}),niter=1000)
                n_iter = n_iter+2000
                success = res_VSF_aSS.success
                VSF_aSS_initial_guess = res_VSF_aSS.x
        dictionary_VSF_aSS = {}
        dictionary_VSF_aSS['fit'] = res_VSF_aSS
        VSF_aSS_fitting_params = pd.DataFrame([res_VSF_aSS.x],columns = ('Ts','Vs','K','T01','S1','Ta1','V01','t1','T02','S2'), index = ['fitted'])
        VSF_aSS_DF_params = pd.concat([VSF_aSS_DF_params,VSF_aSS_fitting_params])
        dictionary_VSF_aSS['fitting params'] = VSF_aSS_DF_params
        #dictionary_VSF_aSS['params tried'] = VSF_aSS_params_tried
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}/VSF_aSS/{k}_{u}_VSF_aSS_t1_neg_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt".format(k=CREEPMETER,u=j), "wb") as tf:
            pickle.dump(dictionary_VSF_aSS,tf)

    else:
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}/VSF_aSS/{k}_{u}_VSF_aSS_t1_neg_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt".format(k=CREEPMETER,u=j), "rb") as tf:
            dictionary_VSF_aSS = pickle.load(tf)
            res_VSF_aSS = dictionary_VSF_aSS['fit']
            VSF_aSS_DF_params = dictionary_VSF_aSS['fitting params']
    return VSF_aSS_DF_params

In [ ]:
dictionary_all_events = {}
if __name__ == '__main__':
    """
    Use multiprocessing to run `test_multi` on each creep event in `dataframes`.
    Currently uses a single worker process (Pool(1)), but can be adjusted for parallelism.
    """
    with Pool(1) as pool:                         # Create a multiprocessing Pool           
        pool.map(test_multi, dataframes)